In [93]:
import re
import requests
from lxml import etree
from user_agent import generate_user_agent
import os
import time
import random

In [14]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys

In [16]:
co = webdriver.ChromeOptions()
# co.add_argument('--headerless')
# co.add_argument('--disable-images')
co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))


# In[287]:


browser=webdriver.Chrome(options=co,service_log_path=os.path.devnull)
browser.get('https://www.dianping.com/')

wait = WebDriverWait(browser,10)

In [18]:
login_button = wait.until(EC.presence_of_element_located((By.XPATH,'//a[@data-click-name="login"]')))

In [19]:
login_button.click()

In [21]:
element = browser.find_element_by_id('J-search-input')

In [22]:
element.send_keys('猫咪寄养')

In [23]:
element.send_keys(Keys.RETURN)

## 普通方法抓取

### 获取店铺列表

In [103]:
ua_ls = ['Opera/9.80(Macintosh;IntelMacOSX10.6.8;U;en)Presto/2.8.131Version/11.11','Mozilla/5.0 (Android 4.4; Tablet; rv:41.0) Gecko/41.0 Firefox/41.0','Mozilla/5.0 (Windows NT x.y; Win64; x64; rv:10.0) Gecko/20100101 Firefox/10.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36']

In [102]:
def get_shop_list(url):
    headers = {'User-Agent':random.choice(ua_ls)}
    r = requests.get(url,headers = headers)
    s = etree.HTML(r.text)
    ls = s.xpath('//div[@id="shop-all-list"]/ul/li')
    shop_ls = []
    for i in range(1,len(ls)+1):
        d = {}
        shop_name = s.xpath(f'//div[@id="shop-all-list"]/ul/li[{i}]//a[@data-click-name="shop_title_click"]/@title')+[999]
        shop_url = s.xpath(f'//div[@id="shop-all-list"]/ul/li[{i}]//a[@data-click-name="shop_title_click"]/@href')+[999]
        shop_addr = s.xpath(f'//div[@id="shop-all-list"]/ul/li[{i}]//span[@class="addr"]/text()')+[999]
        shop_rank = s.xpath(f'//div[@id="shop-all-list"]/ul/li[{i}]//span[@class="sml-rank-stars sml-str50"]/@title')+[999]
        d['name'] = shop_name[0]
        d['url'] = shop_url[0]
        d['addr'] = shop_addr[0]
        d['rank'] = shop_rank[0]
        shop_ls.append(d)
    return shop_ls

In [84]:
shop_ls = []
for page in range(1,16):
    url = f'https://www.dianping.com/search/keyword/1/0_%E7%8C%AB%E5%92%AA%E5%AF%84%E5%85%BB/p{page}'
    print(f'Downloading page {page}...')
    res = get_shop_list(url)
    shop_ls.extend(res)
    time.sleep(2)


In [85]:
len(shop_ls)

74

In [155]:
shop_ls[0]['url']

'http://www.dianping.com/shop/92643866'

## 使用splinter抓取

In [108]:
from splinter import Browser
browser = Browser(driver_name='chrome')

In [109]:
browser.visit('https://www.dianping.com')

In [114]:
search = browser.find_by_id('J-search-input')[0]

In [116]:
search.fill('猫咪寄养')

In [118]:
search_button = browser.find_by_id('J-all-btn')[0]

In [119]:
search_button.click()

In [147]:
browser.windows.current = browser.windows[-1]

In [150]:
next_page = browser.find_by_css('.next')[0]

In [151]:
next_page.click()

In [156]:
browser.visit(shop_ls[0]['url'])

In [166]:
review = browser.find_by_xpath('//a[contains(text(),"更多点评")]')

In [168]:
review[0].click()

In [165]:
if review:
    review[0].click()